In [395]:
from codecs import utf_8_decode, utf_8_encode
from email.encoders import encode_noop
import PyPDF2 as pydf
import unidecode
import pandas as pd
import tabula as read_pdf
from collections import defaultdict
from datetime import datetime

import glob
import os
from pathlib import Path

In [396]:
path = os.getcwd() # get current working directory

c_time = os.path.getctime(path)
# convert creation timestamp into DateTime object
dt_c = datetime.fromtimestamp(c_time)
print('Created on:', dt_c)

Created on: 2022-07-24 20:34:45.551707


Need to get the last file downloaded

In [397]:
time_creation=''
file_name=''

path = "{}\games_info".format(os.getcwd()) # get current working directory

for files in os.listdir(path):
    if 'pdf' in files:
        print(files)
        c_time = os.path.getctime("{0}\\{1}".format(path,files))

        # convert creation timestamp into DateTime object
        dt_c = datetime.fromtimestamp(c_time)
        print('Created on:', dt_c)
        
        if str(dt_c) > time_creation:
            time_creation = str(dt_c)
            file_name = files

Nomeacoes arbitros 10 a 16_06_22.pdf
Created on: 2022-08-01 04:35:30
Nomeacoes arbitros 18 a 24_03_2022.pdf
Created on: 2022-08-09 18:12:09.143886
Nomeacoes arbitros 22 a 28_04_22.pdf
Created on: 2022-08-01 04:35:50
Nomeacoes arbitros 27_05 a 02_06_22 (1).pdf
Created on: 2022-08-09 18:31:03.853142
Nomeacoes arbitros 27_05 a 02_06_22.pdf
Created on: 2022-08-01 06:32:56
Nomeacoes de arbitros - 28_06_2022 (1).pdf
Created on: 2022-08-09 18:09:39.571168
Nomeacoes de arbitros - 28_06_2022.pdf
Created on: 2022-08-01 04:31:34


In [398]:
file_name

'Nomeacoes arbitros 27_05 a 02_06_22 (1).pdf'

In [399]:
file_path = "{0}\\{1}".format(path,file_name)
file_path

'C:\\Users\\gonçalo\\Desktop\\cursos\\testing\\games_info\\Nomeacoes arbitros 27_05 a 02_06_22 (1).pdf'

In [400]:
%%time

pdf_file = open(file_path,'rb')

pdf_reader = pydf.PdfFileReader(pdf_file)

x= pdf_reader.numPages

lista=[]
lista_games=[]

for i in (range(x)):
    pdf_obj=pdf_reader.getPage(i)
    
    text=pdf_obj.extractText()
    text = text.split('\n')
    lista.append(text)

start_date = lista[0][2]
end_date = lista[0][4]

for i in range(len(lista)):
    if i == 0:
        lista_games = [lista[0][5:]]
    else:
        lista_games.append(lista[i])

CPU times: total: 312 ms
Wall time: 305 ms


In [401]:
print("de",start_date,"a",end_date)

de 27-05-2022 a 02-06-2022


THE deleter:

In [402]:
%%time

deletewords = ['jogo','árbitro','função',"página","nota informativa","n.:","data:"]
deletewords = [name.upper() for name in deletewords]

middle_list=[]
new_list_of_games=[]
a=0

for n in range(len(lista_games)):
    for m in range(len(lista_games[n])):
        middle_list.append(lista_games[n][m].upper())
        if m == len(lista_games[n])-1:
            new_list_of_games.append(middle_list.copy())
            middle_list.clear()


for i in range(len(new_list_of_games)):
    for words in new_list_of_games[i]:
        for deleter in deletewords:
            if deleter == words:
                new_list_of_games[i].remove(words)

CPU times: total: 0 ns
Wall time: 0 ns


In [403]:
new_list_of_games

[['CD SENIORES 1ª DIVISÃO - FASE UNICA - SERIE UNICA - 37ª JORNADA',
  'ÁRBITRO',
  '111.00.362.0 PALMELENSE FC - CF TRAFARIA',
  '(149) PAULO BARRADAS',
  'ARBITRO',
  'CAMPO CORNELIO PALMA (RELVADO SINTÉTICO) - PALMELA',
  '(146) ANDRE DUQUE',
  'ARBITRO ASSISTENTE 1',
  '29-05-2022 17:00',
  '(757) RICARDO SARAIVA',
  'ARBITRO ASSISTENTE 2',
  '111.00.363.0 CHARNECA CAPARICA FC - UFC MOITENSE',
  '(535) ANDRE VASQUES',
  'ARBITRO',
  'CAMPO CASSAPO (RELVADO SINTÉTICO) - CHARNECA DA ',
  'CAPARICA',
  '(348) RUBEN BOLEGAS',
  'ARBITRO ASSISTENTE 1',
  '29-05-2022 17:00',
  '(522) JOAO PAIXAO',
  'ARBITRO ASSISTENTE 2',
  '111.00.364.0 UF COMÉRCIO INDÚSTRIA - SEIXAL 1925',
  '(744) DANIEL FERREIRA',
  'ARBITRO',
  'CAMPO BELA VISTA (RELVADO SINTÉTICO) - SETÚBAL',
  '(641) MIGUEL WERGYKOSKI',
  'ARBITRO ASSISTENTE 1',
  '29-05-2022 17:00',
  '(776) DANIEL FIGUEIRA',
  'ARBITRO ASSISTENTE 2',
  '111.00.365.0 C. OLÍMPICO DO MONTIJO, FUT SAD - VASCO ',
  'GAMA AC',
  '(665) GONCALO GIL',


finder 1:

In [404]:
%%time


stopwords = ['seniores','juniores','juvenis','iniciados','benjamins',
             'infantis','feminino','Fut7','CD Juvenil Feminino Fut7',
            ' Sub15','Feminino','Sub22']

competition_list=[]
stopwords=[name.upper() for name in stopwords]

for i in range(len(new_list_of_games)):
    for words in new_list_of_games[i]:
        for stop in stopwords:
            if stop in words:
                competition_list.append(words)


competition_list = [*set(competition_list)]
competition_list

CPU times: total: 15.6 ms
Wall time: 5.6 ms


['TC INICIADOS - FASE UNICA - SERIE "D" - 10ª JORNADA',
 'CD INFANTIS 1º ANO SUB/12 - 2ª FASE FINAL - 2ª. FASE FINAL - 12ª JORNADA',
 'TC INICIADOS - FASE UNICA - SERIE "E" - 10ª JORNADA',
 'TC INICIADOS - FASE UNICA - SERIE "F" - 10ª JORNADA',
 'TC INFANTIS 1º ANO SUB/12 - FASE UNICA - SERIE E - 9ª JORNADA',
 'TC INFANTIS 2º ANO SUB13 - FASE UNICA - SERIE B - 9ª JORNADA',
 'CD INICIADOS 2ª DIVISÃO / CLAW - FASE UNICA - SERIE UNICA - 28ª JORNADA',
 'CD JUNIORES 1ª DIVISÃO - FASE ÚNICA - SERIE UNICA - 28ª JORNADA',
 'TC INFANTIS 2º ANO SUB13 - FASE UNICA - SERIE C - 9ª JORNADA',
 'CD BENJAMINS 1º ANO SUB/10 - 2ª. FASE FINAL - NOVO GRUPO - 11ª JORNADA',
 'T. COMPL. BENJAMINS 2ºANO - FASE ÚNICA - SERIE C - 8ª JORNADA',
 'T. COMPL. BENJAMINS 2ºANO - FASE ÚNICA - SERIE D - 8ª JORNADA',
 'CD INFANTIS 2º ANO SUB/13 - 2ª FASE FINAL - FASE FINAL - 13ª JORNADA',
 'TC JUNIORES - FASE UNICA - SERIE B - 8ª JORNADA',
 'CD SENIORES FUTSAL MASC. - FASE COMPLEMENTAR - SERIE UNICA - 4ª JORNADA',
 'TC IN

In [405]:
print("Number of Competitions",len(competition_list))

Number of Competitions 56


Name finder:

In [406]:
%%time


stopwords_2 = ['Diogo Sousa']

name=[]
stopwords_2=[name.upper() for name in stopwords_2]

for i in range(len(new_list_of_games)):
    for words in new_list_of_games[i]:
        for stop in stopwords_2:
            if stop in words:
                name.append(words)

name
#competition_list = list(dict.fromkeys(competition_list)) #to take out repeated words

CPU times: total: 0 ns
Wall time: 0 ns


['(736) DIOGO SOUSA',
 '(736) DIOGO SOUSA',
 '(736) DIOGO SOUSA',
 '(736) DIOGO SOUSA']

By competition:

In [407]:
%%time


dictionary_of_competitions=defaultdict(list)
dictionary_of_games=defaultdict(list)

competition_checker=''
a=0

for i in range(len(new_list_of_games)):
    for words in new_list_of_games[i]:
        if words in competition_list:
            competition_checker=''
            if words in dictionary_of_competitions.keys():
                competition_checker = competition_checker + words
            else:
                dictionary_of_competitions["{}".format(words)] = []
                competition_checker = competition_checker + words
            
            continue
        
        dictionary_of_competitions[competition_checker].append(words)
dictionary_of_competitions

CPU times: total: 0 ns
Wall time: 0 ns


defaultdict(list,
            {'CD SENIORES 1ª DIVISÃO - FASE UNICA - SERIE UNICA - 37ª JORNADA': ['ÁRBITRO',
              '111.00.362.0 PALMELENSE FC - CF TRAFARIA',
              '(149) PAULO BARRADAS',
              'ARBITRO',
              'CAMPO CORNELIO PALMA (RELVADO SINTÉTICO) - PALMELA',
              '(146) ANDRE DUQUE',
              'ARBITRO ASSISTENTE 1',
              '29-05-2022 17:00',
              '(757) RICARDO SARAIVA',
              'ARBITRO ASSISTENTE 2',
              '111.00.363.0 CHARNECA CAPARICA FC - UFC MOITENSE',
              '(535) ANDRE VASQUES',
              'ARBITRO',
              'CAMPO CASSAPO (RELVADO SINTÉTICO) - CHARNECA DA ',
              'CAPARICA',
              '(348) RUBEN BOLEGAS',
              'ARBITRO ASSISTENTE 1',
              '29-05-2022 17:00',
              '(522) JOAO PAIXAO',
              'ARBITRO ASSISTENTE 2',
              '111.00.364.0 UF COMÉRCIO INDÚSTRIA - SEIXAL 1925',
              '(744) DANIEL FERREIRA',
          

In [408]:
df = pd.DataFrame.from_dict(dictionary_of_competitions,orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,94
CD SENIORES 1ª DIVISÃO - FASE UNICA - SERIE UNICA - 37ª JORNADA,ÁRBITRO,111.00.362.0 PALMELENSE FC - CF TRAFARIA,(149) PAULO BARRADAS,ARBITRO,CAMPO CORNELIO PALMA (RELVADO SINTÉTICO) - PAL...,(146) ANDRE DUQUE,ARBITRO ASSISTENTE 1,29-05-2022 17:00,(757) RICARDO SARAIVA,ARBITRO ASSISTENTE 2,...,(1025) DIOGO MARTINS,ARBITRO ASSISTENTE 2,PÁGINA,1,/,21,N.:,27/05/2022,36/2021-2022/CA/A.F. SETÚBAL,
CD SENIORES 2ª DIVISÃO - FASE UNICA - SERIE UNICA - 33ª JORNADA,ÁRBITRO,112.00.290.0 ACRU TRAB. ZAMBUJALENSE - AC ARRE...,(307) PEDRO FERREIRA,ARBITRO,PARQUE DESPORTIVO DA ACRUT ZAMBUJALENSE (RELVADO,SINTÉTICO) - ZAMBUJAL,(997) PAULO SANTOS,ARBITRO ASSISTENTE 1,29-05-2022 17:00,(1196) TIAGO VIEGAS,...,,None,None,None,None,None,None,None,None,None
CD JUNIORES 1ª DIVISÃO - FASE ÚNICA - SERIE UNICA - 27ª JORNADA,ÁRBITRO,113.00.211.0 UNIÃO SC - GD PESCADORES COSTA CA...,"""A""",(307) PEDRO FERREIRA,ARBITRO,CAMPO MUNICIPAL SÃO SEBASTIÃO (RELVADO SINTÉTI...,SANTIAGO DO CACÉM,(967) CARLOS OLIVEIRA,ARBITRO ASSISTENTE 1,28-05-2022 17:00,...,None,None,None,None,None,None,None,None,None,None
CD JUNIORES 1ª DIVISÃO - FASE ÚNICA - SERIE UNICA - 28ª JORNADA,ÁRBITRO,113.00.221.0 UFC MOITENSE - UF COMÉRCIO INDÚSTRIA,(292) DIOGO TRANCADAS,ARBITRO,JUNCAL DESPORTOS (RELVADO SINTÉTICO) - MOITA,(649) DAVID SOARES,ARBITRO ASSISTENTE 1,29-05-2022 10:30,(999) TIAGO CALHA,ARBITRO ASSISTENTE 2,...,None,None,None,None,None,None,None,None,None,None
CD JUNIORES 1ª DIVISÃO - FASE ÚNICA - SERIE UNICA - 29ª JORNADA,ÁRBITRO,113.00.225.0 GD ALCOCHETENSE - GD FABRIL BARREIRO,(164) JOAO MARQUES,ARBITRO,CAMPO DE JOGOS FRANCISCO CATALÃO (RELVADO,SINTÉTICO) - ALCOCHETE,(654) JOSE FRADE,ARBITRO ASSISTENTE 1,28-05-2022 17:00,(836) TIAGO BAPTISTA,...,None,None,None,None,None,None,None,None,None,None
CD JUNIORES 2ª DIVISÃO - FASE FINAL - SERIE UNICA - 8ª JORNADA,ÁRBITRO,"114.00.022.0 C. OLÍMPICO DO MONTIJO, FUT SAD ""...",ORIENTAL DRAGON FC,(859) LORENA ESTEVES,ARBITRO,CAMPO LIBERDADE (RELVADO SINTÉTICO) - MONTIJO,(720) VASCO MIRA,ARBITRO ASSISTENTE 1,28-05-2022 17:00,(622) MITCHELL GALVAO,...,None,None,None,None,None,None,None,None,None,None
TC JUNIORES - FASE UNICA - SERIE A - 8ª JORNADA,ÁRBITRO,300.02.015.0 GDR O SINDICATO - QUINTAJENSE FC,(734) ALEXANDRE FARINHA,ARBITRO,CAMPO VITOR BATISTA (RELVADO SINTÉTICO) - SETÚBAL,(811) TOMAS DIAS,ARBITRO ASSISTENTE 1,28-05-2022 17:00,(1195) LEONARDO FERNANDES,ARBITRO ASSISTENTE 2,...,None,None,None,None,None,None,None,None,None,None
TC JUNIORES - FASE UNICA - SERIE B - 8ª JORNADA,ÁRBITRO,300.03.022.0 GC CORROIOS - SEIXAL 1925 B,(34) DOMINGOS QUINTELAS,ARBITRO,CAMPO ANTONIO MARQUES PEQUENO (RELVADO SINTÉTI...,CORROIOS,(1225) DIOGO GEIA,ARBITRO ASSISTENTE 1,28-05-2022 17:00,(1017) RICARDO CARVALHO,...,None,None,None,None,None,None,None,None,None,None
CD JUVENIS 2ª DIVISÃO / CLAW - FASE ÚNICA - SERIE UNICA - 28ª JORNADA,ÁRBITRO,116.00.217.0 GD ALFARIM - ALMADA AC,(348) RUBEN BOLEGAS,ARBITRO,"CAMPO ""ALFARIM"" (RELVADO SINTÉTICO) - ALFARIM",(522) JOAO PAIXAO,ARBITRO ASSISTENTE 1,29-05-2022 10:00,(535) ANDRE VASQUES,ARBITRO ASSISTENTE 2,...,36/2021-2022/CA/A.F. SETÚBAL,,None,None,None,None,None,None,None,None
CD JUVENIS 3ª DIVISÃO / CLAW - FASE FINAL - FASE FINAL - 10ª JORNADA,ÁRBITRO,"125.04.028.0 GD FABRIL BARREIRO ""B"" - GD EB D....",(119) HENRIQUE PIRES,ARBITRO,ESTÁDIO ALFREDO SILVA (RELVADO) - LAVRADIO,(985) PEDRO TEODORO,ARBITRO ASSISTENTE 1,28-05-2022 17:00,(1191) IVAN NUNES,ARBITRO ASSISTENTE 2,...,None,None,None,None,None,None,None,None,None,None


Takes futsal out:

In [409]:
football_competitions={}

for i in dictionary_of_competitions.keys():
    if "FUTSAL" in i:
        pass
    else:
        football_competitions[i] = []
        football_competitions[i].append(dictionary_of_competitions[i])

Games with 2 or 1 referee(s):

In [410]:
%%time

referee2_1_list=['benjamins','infantis','feminino','Fut7',
                 'Feminino','CD Juvenil Feminino Fut7',' Sub15']

referee2_1_list=[i.upper() for i in referee2_1_list]

each_game_referee2_1_middle = []
each_game_referee2_1 = []

dict_games_2_1referees={}

for i in football_competitions.keys():
    for age_rank in referee2_1_list:
        if age_rank in i:
            for games_info in football_competitions[i]:
                for data in games_info:
                    each_game_referee2_1_middle.append(data)
                    if "-{} ".format(datetime.today().year) in data:
                        each_game_referee2_1.append(each_game_referee2_1_middle.copy())
                        each_game_referee2_1_middle.clear()
                if len(each_game_referee2_1)>0:
                    dict_games_2_1referees[i] = each_game_referee2_1
                    each_game_referee2_1=[]

CPU times: total: 0 ns
Wall time: 0 ns


In [411]:
dict_games_2_1referees

{'CD INFANTIS 2º ANO SUB/13 FUT9 - 2ª FASE FINAL - 2ª FASE FINAL - 13ª JORNADA': [['ÁRBITRO',
   '750.07.049.0 CD COVA PIEDADE - SEIXAL 1925',
   '(665) GONCALO GIL',
   'ARBITRO',
   'ESTÁDIO MUNIC. JOSÉ MARTINS VIEIRA Nº 2 (RELVADO) - COVA ',
   'DA PIEDADE',
   '(1135) RODRIGO ESTEVES',
   '2º ARBITRO',
   '28-05-2022 10:30'],
  ['750.07.050.0 VITÓRIA FC "A" - FC BARREIRENSE',
   '(744) DANIEL FERREIRA',
   'ARBITRO',
   'CAMPO Nº2 ESTÁDIO DO BONFIM (RELVADO SINTÉTICO) - ',
   'SETUBAL',
   '(776) DANIEL FIGUEIRA',
   '2º ARBITRO',
   '28-05-2022 09:00'],
  ['750.07.051.0 CD OS PELEZINHOS - ESTRELA S. ANDRÉ ACRD',
   '(351) FILIPE COSTA',
   'ARBITRO',
   'CAMPO MUNICIPAL VARZEA (RELVADO SINTÉTICO) - VARZEA - ',
   'SETUBAL',
   '(1196) TIAGO VIEGAS',
   '2º ARBITRO',
   '28-05-2022 10:00'],
  ['750.07.052.0 CD PINHALNOVENSE - VASCO GAMA AC',
   '(726) JORGE CARVALHEIRA DIAS',
   'ARBITRO',
   'CAMPO JOGOS SANTOS JORGE (RELVADO SINTÉTICO) - PINHAL ',
   'NOVO',
   '(791) EURICO VILE

In [412]:
df_2_1_referees = pd.DataFrame.from_dict(dict_games_2_1referees,orient='index')
df_2_1_referees

,0,1,2,3
CD INFANTIS 2º ANO SUB/13 FUT9 - 2ª FASE FINAL - 2ª FASE FINAL - 13ª JORNADA,"[ÁRBITRO, 750.07.049.0 CD COVA PIEDADE - SEIXA...","[750.07.050.0 VITÓRIA FC ""A"" - FC BARREIRENSE,...",[750.07.051.0 CD OS PELEZINHOS - ESTRELA S. AN...,[750.07.052.0 CD PINHALNOVENSE - VASCO GAMA AC...
TC INFANTIS 2º ANO SUB/13 FUT9 - FASE UNICA - SERIE A - 10ª JORNADA,"[ÁRBITRO, 760.01.019.0 AC ALCACERENSE - UNIÃO ...","[760.01.020.0 FC ALVALADENSE - SONHO XXI FC ""A...",None,None
TC INFANTIS 2º ANO SUB/13 FUT9 - FASE UNICA - SERIE B - 10ª JORNADA,"[PÁGINA , 10, / , 21, N.:, 27/05/2022, 36/202...","[760.02.020.0 SONHO XXI FC ""B"" - VITÓRIA FC ""B...",None,None
TC INFANTIS 2º ANO SUB/13 FUT9 - FASE UNICA - SERIE C - 10ª JORNADA,"[ÁRBITRO, 760.03.028.0 EFC AFONSOEIRENSE - GD ...","[760.03.029.0 UFC MOITENSE - SC VINHENSE, (713...",[760.03.030.0 OLÍMPICO MONTIJO - CCD BREJOS AZ...,None
TC INFANTIS 2º ANO SUB/13 FUT9 - FASE UNICA - SERIE D - 10ª JORNADA,"[ÁRBITRO, 760.04.028.0 GD EB D. JOÃO I - GD FA...",[760.04.029.0 AF COL. ATLÂNTICO - AC ARRENTELA...,"[760.04.030.0 GD PESCADORES COSTA CAPARICA ""B""...",None
TC INFANTIS 2º ANO SUB/13 FUT9 - FASE UNICA - SERIE E - 10ª JORNADA,"[PÁGINA , 11, / , 21, N.:, 27/05/2022, 36/202...",[760.05.029.0 AD ALMADA 2015 - GD PESCADORES C...,[760.05.030.0 CF TRAFARIA - CHARNECA CAPARICA ...,None
CD INFANTIS 2º ANO SUB/13 - 2ª FASE FINAL - FASE FINAL - 13ª JORNADA,"[ÁRBITRO, 730.13.049.0 ESTRELA S. ANDRÉ ACRD -...",[730.13.050.0 DREAM TALENTS F.C. - GD LAGAMEÇA...,[730.13.051.0 OLÍMPICO MONTIJO - QUINTAJENSE F...,[730.13.052.0 LEÃO ALTIVO - GD FABRIL BARREIRO...
CD INFANTIS 1º ANO SUB/12 - 2ª FASE FINAL - 2ª. FASE FINAL - 12ª JORNADA,"[PÁGINA , 12, / , 21, N.:, 27/05/2022, 36/202...",None,None,None
CD INFANTIS 1º ANO SUB/12 - 2ª FASE FINAL - 2ª. FASE FINAL - 13ª JORNADA,"[ÁRBITRO, 731.10.049.0 GD ALCOCHETENSE - AMORA...","[731.10.050.0 VITÓRIA FC - LEÃO ALTIVO, (995) ...","[731.10.051.0 FC BARREIRENSE ""A"" - GD FABRIL B...","[731.10.052.0 CD COVA PIEDADE - VASCO GAMA AC,..."
TC INFANTIS 1º ANO SUB/12 - FASE UNICA - SERIE A - 9ª JORNADA,"[ÁRBITRO, 715.01.017.0 SONHO XXI FC - UF COMÉR...",[715.01.018.0 SR CRUZAMENTO PEGÕES - PALMELENS...,None,None


In [413]:
dict_games_2_referees= {}
dict_games_1_referee = dict_games_2_1referees.copy()

for i in dict_games_2_1referees:
    if "FUT9" in i:
        dict_games_2_referees[i] = dict_games_1_referee[i]
        dict_games_1_referee.pop(i)

### Dictionary with 2 referees:

In [414]:
dict_games_2_referees

{'CD INFANTIS 2º ANO SUB/13 FUT9 - 2ª FASE FINAL - 2ª FASE FINAL - 13ª JORNADA': [['ÁRBITRO',
   '750.07.049.0 CD COVA PIEDADE - SEIXAL 1925',
   '(665) GONCALO GIL',
   'ARBITRO',
   'ESTÁDIO MUNIC. JOSÉ MARTINS VIEIRA Nº 2 (RELVADO) - COVA ',
   'DA PIEDADE',
   '(1135) RODRIGO ESTEVES',
   '2º ARBITRO',
   '28-05-2022 10:30'],
  ['750.07.050.0 VITÓRIA FC "A" - FC BARREIRENSE',
   '(744) DANIEL FERREIRA',
   'ARBITRO',
   'CAMPO Nº2 ESTÁDIO DO BONFIM (RELVADO SINTÉTICO) - ',
   'SETUBAL',
   '(776) DANIEL FIGUEIRA',
   '2º ARBITRO',
   '28-05-2022 09:00'],
  ['750.07.051.0 CD OS PELEZINHOS - ESTRELA S. ANDRÉ ACRD',
   '(351) FILIPE COSTA',
   'ARBITRO',
   'CAMPO MUNICIPAL VARZEA (RELVADO SINTÉTICO) - VARZEA - ',
   'SETUBAL',
   '(1196) TIAGO VIEGAS',
   '2º ARBITRO',
   '28-05-2022 10:00'],
  ['750.07.052.0 CD PINHALNOVENSE - VASCO GAMA AC',
   '(726) JORGE CARVALHEIRA DIAS',
   'ARBITRO',
   'CAMPO JOGOS SANTOS JORGE (RELVADO SINTÉTICO) - PINHAL ',
   'NOVO',
   '(791) EURICO VILE

In [415]:
count_games_2_referees = 0 

for keys in dict_games_2_referees:
    for i in dict_games_2_referees[keys]:
        count_games_2_referees+=1

count_games_2_referees

17

### Dictionary with 1 referee:

In [416]:
dict_games_1_referee

{'CD INFANTIS 2º ANO SUB/13 - 2ª FASE FINAL - FASE FINAL - 13ª JORNADA': [['ÁRBITRO',
   '730.13.049.0 ESTRELA S. ANDRÉ ACRD - GD ALFARIM',
   '(1209) JOAO TOME',
   'ARBITRO',
   'COMPLEXO DESPORTIVO CAMPO Nº 2 ESTRELA STº ANDRÉ (RELVADO SINTÉTICO) - VILA NOVA STº ANDRÉ',
   '28-05-2022 10:00'],
  ['730.13.050.0 DREAM TALENTS F.C. - GD LAGAMEÇAS',
   '(683) BRUNO F. SILVA',
   'ARBITRO',
   'CAMPO BOA HORA (RELVADO SINTÉTICO) - ARRENTELA',
   '28-05-2022 11:00'],
  ['730.13.051.0 OLÍMPICO MONTIJO - QUINTAJENSE FC',
   '(591) PEDRO CARDADOR',
   'ARBITRO',
   'CAMPO LIBERDADE (RELVADO SINTÉTICO) - MONTIJO',
   '28-05-2022 10:30'],
  ['730.13.052.0 LEÃO ALTIVO - GD FABRIL BARREIRO',
   '(119) HENRIQUE PIRES',
   'ARBITRO',
   'CAMPO ESCOLA EB 2+3 DE VALE MILHAÇOS (RELVADO SINTÉTICO) - SEIXAL',
   '28-05-2022 11:00']],
 'CD INFANTIS 1º ANO SUB/12 - 2ª FASE FINAL - 2ª. FASE FINAL - 12ª JORNADA': [['PÁGINA ',
   '12',
   ' / ',
   '21',
   'N.:',
   '27/05/2022',
   '36/2021-2022/CA/A.F. S

In [417]:
count_games_1_referee = 0 

for keys in dict_games_1_referee:
    for i in dict_games_1_referee[keys]:
        count_games_1_referee+=1

count_games_1_referee

54

### With 3 referees:

In [418]:
%%time


referee3_list=['seniores','juniores','juvenis','iniciados','sub22']
referee3_list=[i.upper() for i in referee3_list]

each_game_referee3_middle = []
each_game_referee3 = []

dict_games_3referees={}

#games with 3 referees:
for i in football_competitions.keys():
    for age_rank in referee3_list:
        if age_rank in i:
            for games_info in football_competitions[i]:
                for data in games_info:
                    each_game_referee3_middle.append(data)
                    if "ARBITRO ASSISTENTE 2" == data:
                        each_game_referee3.append(each_game_referee3_middle.copy())
                        each_game_referee3_middle.clear()
                if len(each_game_referee3)>0:
                    dict_games_3referees[i] = each_game_referee3
                    each_game_referee3=[]

CPU times: total: 0 ns
Wall time: 0 ns


In [419]:
count_games_3_referee = 0 

for keys in dict_games_3referees:
    for i in dict_games_3referees[keys]:
        count_games_3_referee+=1

count_games_3_referee

66

In [420]:
%%time

def referee_1():
    
    competition_keys_checker = ''
    codigo_jogo = defaultdict(list)

    jogos_info_1_referee = {}
    i=0

    for competition_keys in dict_games_1_referee.keys():
        for competition_games in dict_games_1_referee[competition_keys]:
            for list_info in competition_games:
                i+=1
                
                if list_info == 'ARBITRO':
                    if i == 4 and competition_games[i-4] == 'ÁRBITRO': 
                        codigo = competition_games[i-3].split(' ')[0]
                        equipas = competition_games[i-3].replace(codigo,'')
                        arbitro = competition_games[i-2].split(' ')
                        arbitro =' '.join(arbitro[1:])
                        n_arbitro = competition_games[i-2].split(' ')[0]
                        dia = competition_games[-1].split(' ')[0]
                        hora = competition_games[-1].split(' ')[1]
                        
                        try:
                            campo = competition_games[i].split('-')[0]
                            local = competition_games[i].split('-')[1]
                        except:
                            campo = competition_games[i]

                        codigo_jogo[codigo] = [] 
                        
                        codigo_jogo[codigo].append(equipas)
                        codigo_jogo[codigo].append(arbitro)
                        codigo_jogo[codigo].append(n_arbitro)
                        codigo_jogo[codigo].append(campo)
                        codigo_jogo[codigo].append(local)
                        codigo_jogo[codigo].append(dia)
                        codigo_jogo[codigo].append(hora)
                        
                        continue

                    if i == 4 and competition_games[i-4] != 'ÁRBITRO':
                        codigo = competition_games[i-3].split(' ')[0]
                        equipas = competition_games[i-3].replace(codigo,'')
                        arbitro = competition_games[i-2].split(' ')
                        arbitro =' '.join(arbitro[1:])
                        n_arbitro = competition_games[i-2].split(' ')[0]
                        dia = competition_games[-1].split(' ')[0]
                        hora = competition_games[-1].split(' ')[1]
                        
                        
                        try:
                            campo = competition_games[i].split('-')[0]
                            local = competition_games[i].split('-')[1]
                        except:
                            campo = competition_games[i]

                        codigo_jogo[codigo] = [] 
                        
                        codigo_jogo[codigo].append(equipas)
                        codigo_jogo[codigo].append(arbitro)
                        codigo_jogo[codigo].append(n_arbitro)
                        codigo_jogo[codigo].append(campo)
                        codigo_jogo[codigo].append(local)
                        codigo_jogo[codigo].append(dia)
                        codigo_jogo[codigo].append(hora)
                        
                        continue
                        
                    if i == 5:
                        codigo = competition_games[i-3].split(' ')[0]
                        equipas = competition_games[i-3].replace(codigo,'')
                        arbitro = competition_games[i-2].split(' ')
                        arbitro =' '.join(arbitro[1:])
                        n_arbitro = competition_games[i-2].split(' ')[0]
                        dia = competition_games[-1].split(' ')[0]
                        hora = competition_games[-1].split(' ')[1]
                    
                        try:
                            campo = competition_games[i].split('-')[0]
                            local = competition_games[i].split('-')[1]
                        except:
                            campo = competition_games[i]
                        
                        codigo_jogo[codigo] = []
                        
                        codigo_jogo[codigo].append(equipas)
                        codigo_jogo[codigo].append(arbitro)
                        codigo_jogo[codigo].append(n_arbitro)
                        codigo_jogo[codigo].append(campo)
                        codigo_jogo[codigo].append(local)
                        codigo_jogo[codigo].append(dia)
                        codigo_jogo[codigo].append(hora)
                        
                        continue
                        
                    else:
                        codigo = competition_games[i-3].split(' ')[0]
                        equipas = competition_games[i-3].replace(codigo,'')
                        arbitro = competition_games[i-2].split(' ')
                        arbitro =' '.join(arbitro[1:])
                        n_arbitro = competition_games[i-2].split(' ')[0]
                        dia = competition_games[-1].split(' ')[0]
                        hora = competition_games[-1].split(' ')[1]

                        try:
                            campo = competition_games[i].split('-')[0]
                            local = competition_games[i].split('-')[1]
                        except:
                            campo = competition_games[i]

                        codigo_jogo[codigo] = [] 
                        codigo_jogo[codigo].append(equipas)
                        codigo_jogo[codigo].append(arbitro)
                        codigo_jogo[codigo].append(n_arbitro)
                        codigo_jogo[codigo].append(campo)
                        codigo_jogo[codigo].append(local)
                        codigo_jogo[codigo].append(dia)
                        codigo_jogo[codigo].append(hora)
                        

            i=0
        jogos_info_1_referee[competition_keys] = codigo_jogo
        codigo_jogo = defaultdict(list)
    
    return jogos_info_1_referee

referee_1()

CPU times: total: 0 ns
Wall time: 0 ns


{'CD INFANTIS 2º ANO SUB/13 - 2ª FASE FINAL - FASE FINAL - 13ª JORNADA': defaultdict(list,
             {'730.13.049.0': [' ESTRELA S. ANDRÉ ACRD - GD ALFARIM',
               'JOAO TOME',
               '(1209)',
               'COMPLEXO DESPORTIVO CAMPO Nº 2 ESTRELA STº ANDRÉ (RELVADO SINTÉTICO) ',
               ' VILA NOVA STº ANDRÉ',
               '28-05-2022',
               '10:00'],
              '730.13.050.0': [' DREAM TALENTS F.C. - GD LAGAMEÇAS',
               'BRUNO F. SILVA',
               '(683)',
               'CAMPO BOA HORA (RELVADO SINTÉTICO) ',
               ' ARRENTELA',
               '28-05-2022',
               '11:00'],
              '730.13.051.0': [' OLÍMPICO MONTIJO - QUINTAJENSE FC',
               'PEDRO CARDADOR',
               '(591)',
               'CAMPO LIBERDADE (RELVADO SINTÉTICO) ',
               ' MONTIJO',
               '28-05-2022',
               '10:30'],
              '730.13.052.0': [' LEÃO ALTIVO - GD FABRIL BARREIRO',
            

In [421]:
df_1referee = pd.DataFrame.from_dict(referee_1(),orient='index')
df_1referee.columns

Index(['730.13.049.0', '730.13.050.0', '730.13.051.0', '730.13.052.0',
       '731.10.048.0', '731.10.049.0', '731.10.050.0', '731.10.051.0',
       '731.10.052.0', '715.01.017.0', '715.01.018.0', '715.02.017.0',
       '715.02.018.0', '715.03.017.0', '715.03.018.0', '715.04.017.0',
       '715.04.018.0', '715.05.017.0', '715.05.018.0', '715.06.025.0', 'COSTA',
       '715.06.027.0', '740.00.017.0', '740.00.018.0', '740.01.025.0',
       '740.01.026.0', '740.01.027.0', '740.02.025.0', '740.02.026.0',
       '740.02.027.0', 'ZAMBUJALENSE', '740.03.026.0', '740.03.027.0',
       '732.10.046.0', '732.10.047.0', '732.10.048.0', '733.06.041.0',
       '733.06.042.0', '733.06.043.0', '733.06.044.0', '714.01.016.0',
       '714.03.015.0', '714.04.015.0', '717.01.020.0', '717.06.024.0',
       '717.07.026.0', '901.00.020.1', '901.00.021.0', '901.00.022.0',
       '901.00.023.0', '470.00.016.0', '470.00.017.0', '470.00.018.0'],
      dtype='object')

In [422]:
len(df_1referee.columns)

53

In [423]:
count_games_1_referee

54

Error:

In [424]:
count_error = 0

for i in df_1referee.columns:
    if '.' in i:
        pass
    else:
        count_error+=1

try:
    accuracy_1referees = round((1 - count_error / len(df_1referee.columns))*100,2)
    print('Accuracy:', accuracy_1referees,'%')
except Exception as e:
    print(e)

Accuracy: 96.23 %


In [425]:
dict_games_2_referees

{'CD INFANTIS 2º ANO SUB/13 FUT9 - 2ª FASE FINAL - 2ª FASE FINAL - 13ª JORNADA': [['ÁRBITRO',
   '750.07.049.0 CD COVA PIEDADE - SEIXAL 1925',
   '(665) GONCALO GIL',
   'ARBITRO',
   'ESTÁDIO MUNIC. JOSÉ MARTINS VIEIRA Nº 2 (RELVADO) - COVA ',
   'DA PIEDADE',
   '(1135) RODRIGO ESTEVES',
   '2º ARBITRO',
   '28-05-2022 10:30'],
  ['750.07.050.0 VITÓRIA FC "A" - FC BARREIRENSE',
   '(744) DANIEL FERREIRA',
   'ARBITRO',
   'CAMPO Nº2 ESTÁDIO DO BONFIM (RELVADO SINTÉTICO) - ',
   'SETUBAL',
   '(776) DANIEL FIGUEIRA',
   '2º ARBITRO',
   '28-05-2022 09:00'],
  ['750.07.051.0 CD OS PELEZINHOS - ESTRELA S. ANDRÉ ACRD',
   '(351) FILIPE COSTA',
   'ARBITRO',
   'CAMPO MUNICIPAL VARZEA (RELVADO SINTÉTICO) - VARZEA - ',
   'SETUBAL',
   '(1196) TIAGO VIEGAS',
   '2º ARBITRO',
   '28-05-2022 10:00'],
  ['750.07.052.0 CD PINHALNOVENSE - VASCO GAMA AC',
   '(726) JORGE CARVALHEIRA DIAS',
   'ARBITRO',
   'CAMPO JOGOS SANTOS JORGE (RELVADO SINTÉTICO) - PINHAL ',
   'NOVO',
   '(791) EURICO VILE

In [426]:
competition_keys_checker = ''
codigo_jogo = defaultdict(list)

jogos_info_2_referee = {}
i=0

for competition_keys in dict_games_2_referees.keys():
    for competition_games in dict_games_2_referees[competition_keys]:
        for list_info in competition_games:
            i+=1
            if list_info == 'ARBITRO':
                campo = competition_games[i]
                print(campo)
                local = competition_games[i+1]
                print(local)
                
                print("\n")
                campo_usar = competition_games[i].split('-')[0].split('(')[0]
                print('CAMPO:',campo_usar)
                    
                if '-' in competition_games[i]:
                    
                    if competition_games[i].split('-')[1].split(' ')[-1]:
                        local = competition_games[i].split('-')[1]
                        print('LOCAL:',local)
   
                    if not competition_games[i].split('-')[1].split(' ')[-1]:
                        local_1 = competition_games[i].split('-')[1]
                        local_2 = competition_games[i+1]
                        local = local_1+local_2
                        print('LOCAL:',local)
                else:
                    try:
                        local = competition_games[i+1].split('-')[1]
                        print('LOCAL:',local)
                    except:
                        local = None
                        print('LOCAL:',local)
        print("*"*20)
        i=0

ESTÁDIO MUNIC. JOSÉ MARTINS VIEIRA Nº 2 (RELVADO) - COVA 
DA PIEDADE


CAMPO: ESTÁDIO MUNIC. JOSÉ MARTINS VIEIRA Nº 2 
LOCAL:  COVA DA PIEDADE
********************
CAMPO Nº2 ESTÁDIO DO BONFIM (RELVADO SINTÉTICO) - 
SETUBAL


CAMPO: CAMPO Nº2 ESTÁDIO DO BONFIM 
LOCAL:  SETUBAL
********************
CAMPO MUNICIPAL VARZEA (RELVADO SINTÉTICO) - VARZEA - 
SETUBAL


CAMPO: CAMPO MUNICIPAL VARZEA 
LOCAL:  VARZEA SETUBAL
********************
CAMPO JOGOS SANTOS JORGE (RELVADO SINTÉTICO) - PINHAL 
NOVO


CAMPO: CAMPO JOGOS SANTOS JORGE 
LOCAL:  PINHAL NOVO
********************
ESTADIO MUNICIPAL ALCACER SAL (RELVADO SINTÉTICO) - 
ALCACER DO SAL


CAMPO: ESTADIO MUNICIPAL ALCACER SAL 
LOCAL:  ALCACER DO SAL
********************
CAMPO DE JOGOS - MUNICIPAL DE  ALVALADE (RELVADO 
SINTÉTICO) - ALVALADE DO SADO


CAMPO: CAMPO DE JOGOS 
LOCAL:  MUNICIPAL DE  ALVALADE (RELVADO SINTÉTICO) - ALVALADE DO SADO
********************
CAMPO CORNELIO PALMA (RELVADO SINTÉTICO) - PALMELA
(1179) IVAN MARTINS


CAMPO

In [427]:
%%time

def referee_2():
    
    competition_keys_checker = ''
    codigo_jogo = defaultdict(list)

    jogos_info_2_referee = {}
    i=0

    for competition_keys in dict_games_2_referees.keys():
        for competition_games in dict_games_2_referees[competition_keys]:
            for list_info in competition_games:
                i+=1
                if list_info == 'ARBITRO':
                    if i == 4 and competition_games[i-4] == 'ÁRBITRO': 
                        #dados do jogo:
                        codigo = competition_games[i-3].split(' ')[0]
                        equipas = competition_games[i-3].replace(codigo,'')
                        arbitro = competition_games[i-2].split(' ')
                        arbitro =' '.join(arbitro[1:])
                        n_arbitro = competition_games[i-2].split(' ')[0]
                        arbitro_2 = competition_games[-3].split(' ')
                        arbitro_2 = ' '.join(arbitro_2[1:])
                        n_arbitro_2 = competition_games[-3].split(' ')[0]
                        dia = competition_games[-1].split(' ')[0]
                        hora = competition_games[-1].split(' ')[1]

                        campo = competition_games[i].split('-')[0].split('(')[0]
                        if '-' in competition_games[i]:
                            if competition_games[i].split('-')[1].split(' ')[-1]:
                                local = competition_games[i].split('-')[1]

                            if not competition_games[i].split('-')[1].split(' ')[-1]:
                                local_1 = competition_games[i].split('-')[1]
                                local_2 = competition_games[i+1]
                                local = local_1+local_2
                        else:
                            try:
                                local = competition_games[i+1].split('-')[1]
                            except:
                                local = None


                        codigo_jogo[codigo] = []

                        #appends:
                        codigo_jogo[codigo].append(equipas)
                        codigo_jogo[codigo].append(arbitro)
                        codigo_jogo[codigo].append(n_arbitro)
                        codigo_jogo[codigo].append(arbitro_2)
                        codigo_jogo[codigo].append(n_arbitro_2)
                        codigo_jogo[codigo].append(campo)
                        codigo_jogo[codigo].append(local)
                        codigo_jogo[codigo].append(dia)
                        codigo_jogo[codigo].append(hora)   

                        continue

                    if i == 4 and competition_games[i-4] != 'ÁRBITRO':
                        #dados do jogo:
                        codigo = competition_games[i-4].split(' ')[0]
                        equipas = competition_games[i-4].replace(codigo,'')
                        arbitro = competition_games[i-2].split(' ')
                        arbitro =' '.join(arbitro[1:])
                        n_arbitro = competition_games[i-2].split(' ')[0]
                        arbitro_2 = competition_games[-3].split(' ')
                        arbitro_2 = ' '.join(arbitro_2[1:])
                        n_arbitro_2 = competition_games[-3].split(' ')[0]
                        dia = competition_games[-1].split(' ')[0]
                        hora = competition_games[-1].split(' ')[1]

                        codigo_jogo[codigo] = []

                        campo = competition_games[i].split('-')[0].split('(')[0]
                        if '-' in competition_games[i]:

                            if competition_games[i].split('-')[1].split(' ')[-1]:
                                local = competition_games[i].split('-')[1]

                            if not competition_games[i].split('-')[1].split(' ')[-1]:
                                local_1 = competition_games[i].split('-')[1]
                                local_2 = competition_games[i+1]
                                local = local_1+local_2
                        else:
                            try:
                                local = competition_games[i+1].split('-')[1]
                            except:
                                local = None

                        #appends:
                        codigo_jogo[codigo].append(equipas)
                        codigo_jogo[codigo].append(arbitro)
                        codigo_jogo[codigo].append(n_arbitro)
                        codigo_jogo[codigo].append(arbitro_2)
                        codigo_jogo[codigo].append(n_arbitro_2)
                        codigo_jogo[codigo].append(campo)
                        codigo_jogo[codigo].append(local)
                        codigo_jogo[codigo].append(dia)
                        codigo_jogo[codigo].append(hora)   

                        continue

                    if i == 5 and competition_games[i-5] == 'ÁRBITRO':
                        #dados do jogo:
                        codigo = competition_games[i-4].split(' ')[0]
                        equipas = competition_games[i-4].replace(codigo,'')
                        arbitro = competition_games[i-2].split(' ')
                        arbitro =' '.join(arbitro[1:])
                        n_arbitro = competition_games[i-2].split(' ')[0]
                        arbitro_2 = competition_games[-3].split(' ')
                        arbitro_2 = ' '.join(arbitro_2[1:])
                        n_arbitro_2 = competition_games[-3].split(' ')[0]
                        dia = competition_games[-1].split(' ')[0]
                        hora = competition_games[-1].split(' ')[1]

                        codigo_jogo[codigo] = []

                        campo = competition_games[i].split('-')[0].split('(')[0]
                        if '-' in competition_games[i]:

                            if competition_games[i].split('-')[1].split(' ')[-1]:
                                local = competition_games[i].split('-')[1]

                            if not competition_games[i].split('-')[1].split(' ')[-1]:
                                local_1 = competition_games[i].split('-')[1]
                                local_2 = competition_games[i+1]
                                local = local_1+local_2
                        else:
                            try:
                                local = competition_games[i+1].split('-')[1]
                            except:
                                local = None

                        #appends:
                        codigo_jogo[codigo].append(equipas)
                        codigo_jogo[codigo].append(arbitro)
                        codigo_jogo[codigo].append(n_arbitro)
                        codigo_jogo[codigo].append(arbitro_2)
                        codigo_jogo[codigo].append(n_arbitro_2)
                        codigo_jogo[codigo].append(campo)
                        codigo_jogo[codigo].append(local)
                        codigo_jogo[codigo].append(dia)
                        codigo_jogo[codigo].append(hora)   


                        continue

                    else:
                        codigo = competition_games[i-3].split(' ')[0]
                        equipas = competition_games[i-3].replace(codigo,'')
                        arbitro = competition_games[i-2].split(' ')
                        arbitro =' '.join(arbitro[1:])
                        n_arbitro = competition_games[i-2].split(' ')[0]
                        arbitro_2 = competition_games[-3].split(' ')
                        arbitro_2 = ' '.join(arbitro_2[1:])
                        n_arbitro_2 = competition_games[-3].split(' ')[0]
                        dia = competition_games[-1].split(' ')[0]
                        hora = competition_games[-1].split(' ')[1]

                        codigo_jogo[codigo] = []  

                        campo = competition_games[i].split('-')[0].split('(')[0]
                        if '-' in competition_games[i]:

                            if competition_games[i].split('-')[1].split(' ')[-1]:
                                local = competition_games[i].split('-')[1]

                            if not competition_games[i].split('-')[1].split(' ')[-1]:
                                local_1 = competition_games[i].split('-')[1]
                                local_2 = competition_games[i+1]
                                local = local_1+local_2
                        else:
                            try:
                                local = competition_games[i+1].split('-')[1]
                            except:
                                local = None

                        #appends:
                        codigo_jogo[codigo].append(equipas)
                        codigo_jogo[codigo].append(arbitro)
                        codigo_jogo[codigo].append(n_arbitro)
                        codigo_jogo[codigo].append(arbitro_2)
                        codigo_jogo[codigo].append(n_arbitro_2)
                        codigo_jogo[codigo].append(campo)
                        codigo_jogo[codigo].append(local)
                        codigo_jogo[codigo].append(dia)
                        codigo_jogo[codigo].append(hora)                    


            i=0
        jogos_info_2_referee[competition_keys] = codigo_jogo
        codigo_jogo = defaultdict(list)
    
    return jogos_info_2_referee

referee_2()

CPU times: total: 0 ns
Wall time: 0 ns


{'CD INFANTIS 2º ANO SUB/13 FUT9 - 2ª FASE FINAL - 2ª FASE FINAL - 13ª JORNADA': defaultdict(list,
             {'750.07.049.0': [' CD COVA PIEDADE - SEIXAL 1925',
               'GONCALO GIL',
               '(665)',
               'RODRIGO ESTEVES',
               '(1135)',
               'ESTÁDIO MUNIC. JOSÉ MARTINS VIEIRA Nº 2 ',
               ' COVA DA PIEDADE',
               '28-05-2022',
               '10:30'],
              '750.07.050.0': [' VITÓRIA FC "A" - FC BARREIRENSE',
               'DANIEL FERREIRA',
               '(744)',
               'DANIEL FIGUEIRA',
               '(776)',
               'CAMPO Nº2 ESTÁDIO DO BONFIM ',
               ' SETUBAL',
               '28-05-2022',
               '09:00'],
              '750.07.051.0': [' CD OS PELEZINHOS - ESTRELA S. ANDRÉ ACRD',
               'FILIPE COSTA',
               '(351)',
               'TIAGO VIEGAS',
               '(1196)',
               'CAMPO MUNICIPAL VARZEA ',
               ' VARZEA SETUBAL',
 

In [428]:
df_2referee = pd.DataFrame.from_dict(referee_2(),orient='index')
df_2referee.columns

Index(['750.07.049.0', '750.07.050.0', '750.07.051.0', '750.07.052.0',
       '760.01.019.0', '760.01.020.0', '760.02.019.0', '760.02.020.0',
       '760.03.028.0', '760.03.029.0', '760.03.030.0', '760.04.028.0',
       '760.04.029.0', '760.04.030.0', '760.05.028.0', '760.05.029.0',
       '760.05.030.0'],
      dtype='object')

In [429]:
len(df_2referee.columns)

17

In [430]:
count_games_2_referees

17

In [431]:
count_error = 0

for i in df_2referee.columns:
    if '.' in i:
        pass
    else:
        count_error+=1

try:
    accuracy_2referees = round((1 - count_error / len(df_2referee.columns))*100,2)   
    print('Accuracy:', accuracy_2referees,'%')
except Exception as e:
    print(e)

Accuracy: 100.0 %


In [432]:
dict_games_3referees

{'CD SENIORES 1ª DIVISÃO - FASE UNICA - SERIE UNICA - 37ª JORNADA': [['ÁRBITRO',
   '111.00.362.0 PALMELENSE FC - CF TRAFARIA',
   '(149) PAULO BARRADAS',
   'ARBITRO',
   'CAMPO CORNELIO PALMA (RELVADO SINTÉTICO) - PALMELA',
   '(146) ANDRE DUQUE',
   'ARBITRO ASSISTENTE 1',
   '29-05-2022 17:00',
   '(757) RICARDO SARAIVA',
   'ARBITRO ASSISTENTE 2'],
  ['111.00.363.0 CHARNECA CAPARICA FC - UFC MOITENSE',
   '(535) ANDRE VASQUES',
   'ARBITRO',
   'CAMPO CASSAPO (RELVADO SINTÉTICO) - CHARNECA DA ',
   'CAPARICA',
   '(348) RUBEN BOLEGAS',
   'ARBITRO ASSISTENTE 1',
   '29-05-2022 17:00',
   '(522) JOAO PAIXAO',
   'ARBITRO ASSISTENTE 2'],
  ['111.00.364.0 UF COMÉRCIO INDÚSTRIA - SEIXAL 1925',
   '(744) DANIEL FERREIRA',
   'ARBITRO',
   'CAMPO BELA VISTA (RELVADO SINTÉTICO) - SETÚBAL',
   '(641) MIGUEL WERGYKOSKI',
   'ARBITRO ASSISTENTE 1',
   '29-05-2022 17:00',
   '(776) DANIEL FIGUEIRA',
   'ARBITRO ASSISTENTE 2'],
  ['111.00.365.0 C. OLÍMPICO DO MONTIJO, FUT SAD - VASCO ',
   'G

In [449]:
i=0
jogos_info_3_referee={}
codigo_jogo = defaultdict(list)

for competition_keys in dict_games_3referees.keys():
    for competition_games in dict_games_3referees[competition_keys]:
        for list_info in competition_games:
            i+=1
            if list_info == 'ARBITRO':
                if i == 4 and competition_games[i-4] == 'ÁRBITRO':
                    codigo = competition_games[i-3].split(' ')[0]
                    equipas = competition_games[i-3].replace(codigo,'')
                    arbitro = competition_games[i-2].split(' ')
                    arbitro =' '.join(arbitro[1:])
                    n_arbitro = competition_games[i-2].split(' ')[0]         
                    
                    codigo_jogo[codigo] = []

                    
                    campo = competition_games[i].split('-')[0].split('(')[0]
                    if '-' in competition_games[i]:
                        if competition_games[i].split('-')[1].split(' ')[-1]:
                            local = competition_games[i].split('-')[1]

                        if not competition_games[i].split('-')[1].split(' ')[-1]:
                            local_1 = competition_games[i].split('-')[1]
                            local_2 = competition_games[i+1]
                            local = local_1+local_2
                    else:
                        try:
                            local = competition_games[i+1].split('-')[1]
                        except:
                            local = None
                            
                    codigo_jogo[codigo].append(equipas)
                    codigo_jogo[codigo].append(arbitro)
                    codigo_jogo[codigo].append(n_arbitro)
                    
                    
                if i == 4 and competition_games[i-4] != 'ÁRBITRO':
                    codigo = competition_games[i-4].split(' ')[0]
                    equipas = competition_games[i-4].replace(codigo,'')
                    arbitro = competition_games[i-2].split(' ')
                    arbitro =' '.join(arbitro[1:])

                    codigo_jogo[codigo] = []

                    campo = competition_games[i].split('-')[0].split('(')[0]
                    if '-' in competition_games[i]:
                        if competition_games[i].split('-')[1].split(' ')[-1]:
                            local = competition_games[i].split('-')[1]

                        if not competition_games[i].split('-')[1].split(' ')[-1]:
                            local_1 = competition_games[i].split('-')[1]
                            local_2 = competition_games[i+1]
                            local = local_1+local_2
                    else:
                        try:
                            local = competition_games[i+1].split('-')[1]
                        except:
                            local = None
                                
                    codigo_jogo[codigo].append(equipas)
                    codigo_jogo[codigo].append(arbitro)
                    codigo_jogo[codigo].append(n_arbitro)
                        
                if i == 5 and competition_games[i-5] == 'ÁRBITRO':
                    codigo = competition_games[i-4].split(' ')[0]
                    equipas = competition_games[i-4].replace(codigo,'')
                    arbitro = competition_games[i-2].split(' ')
                    arbitro =' '.join(arbitro[1:])
                    n_arbitro = competition_games[i-2].split(' ')[0]
                    
                    codigo_jogo[codigo] = []

                    campo = competition_games[i].split('-')[0].split('(')[0]
                    if '-' in competition_games[i]:

                        if competition_games[i].split('-')[1].split(' ')[-1]:
                            local = competition_games[i].split('-')[1]

                        if not competition_games[i].split('-')[1].split(' ')[-1]:
                            local_1 = competition_games[i].split('-')[1]
                            local_2 = competition_games[i+1]
                            local = local_1+local_2
                    else:
                        try:
                            local = competition_games[i+1].split('-')[1]
                        except:
                            local = None

                    #appends:
                    codigo_jogo[codigo].append(equipas)
                    codigo_jogo[codigo].append(arbitro)
                    codigo_jogo[codigo].append(n_arbitro)
                    
                else:
                    codigo = competition_games[i-3].split(' ')[0]
                    equipas = competition_games[i-3].replace(codigo,'')
                    arbitro = competition_games[i-2].split(' ')
                    arbitro =' '.join(arbitro[1:])
                    n_arbitro = competition_games[i-2].split(' ')[0]
                    
                    codigo_jogo[codigo] = []  

                    campo = competition_games[i].split('-')[0].split('(')[0]
                    if '-' in competition_games[i]:

                        if competition_games[i].split('-')[1].split(' ')[-1]:
                            local = competition_games[i].split('-')[1]

                        if not competition_games[i].split('-')[1].split(' ')[-1]:
                            local_1 = competition_games[i].split('-')[1]
                            local_2 = competition_games[i+1]
                            local = local_1+local_2
                    else:
                        try:
                            local = competition_games[i+1].split('-')[1]
                        except:
                            local = None

                    #appends:
                    codigo_jogo[codigo].append(equipas)
                    codigo_jogo[codigo].append(arbitro)
                    codigo_jogo[codigo].append(n_arbitro)                    
                    
                    
        i=0
    jogos_info_3_referee[competition_keys] = codigo_jogo
    codigo_jogo = defaultdict(list) 
    
jogos_info_3_referee

{'CD SENIORES 1ª DIVISÃO - FASE UNICA - SERIE UNICA - 37ª JORNADA': defaultdict(list,
             {'111.00.362.0': [' PALMELENSE FC - CF TRAFARIA',
               'PAULO BARRADAS',
               '(149)'],
              '111.00.363.0': [' CHARNECA CAPARICA FC - UFC MOITENSE',
               'ANDRE VASQUES',
               '(535)'],
              '111.00.364.0': [' UF COMÉRCIO INDÚSTRIA - SEIXAL 1925',
               'DANIEL FERREIRA',
               '(744)'],
              '111.00.365.0': [' C. OLÍMPICO DO MONTIJO, FUT SAD - VASCO ',
               'GONCALO GIL',
               '(744)'],
              'GAMA': [' AC', 'GONCALO GIL', '(665)'],
              '111.00.366.0': [' UNIÃO SC - GD ALFARIM',
               'HENRIQUE PIRES',
               '(119)'],
              '111.00.367.0': [' GD ALCOCHETENSE "A" - GD PESCADORES ',
               'BRUNO SILVA',
               '(119)'],
              'COSTA': [' CAPARICA', 'BRUNO SILVA', '(413)'],
              '111.00.368.0': [' CDR ÁGUAS MO

In [451]:
df_3referee = pd.DataFrame.from_dict(jogos_info_3_referee,orient='index')
df_3referee.columns

Index(['111.00.362.0', '111.00.363.0', '111.00.364.0', '111.00.365.0', 'GAMA',
       '111.00.366.0', '111.00.367.0', 'COSTA', '111.00.368.0', '111.00.369.0',
       '111.00.370.0', 'GRANDOLENSE', '112.00.290.0', '112.00.291.0',
       '112.00.292.0', '112.00.293.0', 'ACRD', '112.00.294.0', '112.00.295.0',
       '112.00.296.0', '112.00.297.0', '"A"', '113.00.221.0', '113.00.225.0',
       '114.00.022.0', '114.00.023.0', '114.00.024.0', '300.02.015.0',
       '300.02.016.0', '300.03.022.0', '300.03.023.0', '300.03.024.0', 'FC',
       '116.00.217.0', '116.00.218.0', '116.00.219.0', '116.00.220.0',
       '116.00.221.0', '116.00.222.0', '116.00.223.0', '"B"', '116.00.224.0',
       '125.04.028.0', '125.04.029.0', '125.04.030.0', '117.00.199.0',
       '118.00.217.0', '118.00.218.0', '118.00.219.0', '118.00.220.0',
       '118.00.221.0', '118.00.222.0', '118.00.223.0', '118.00.224.0',
       '123.03.049.0', '123.03.050.0', '123.03.051.0', '123.03.052.0',
       '119.10.019.0', '119.10.02

In [452]:
count_error = 0

for i in df_3referee.columns:
    if '.' in i:
        pass
    else:
        count_error+=1

try:
    accuracy_3referees = round((1 - count_error / len(df_3referee.columns))*100,2)   
    print('Accuracy:', accuracy_3referees,'%')
except Exception as e:
    print(e)

Accuracy: 88.89 %


In [454]:
(accuracy_1referees + accuracy_2referees + accuracy_3referees) / 3

95.04